<a href="https://colab.research.google.com/github/Elbereth-Elentari/Book_recommender/blob/master/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [2]:
cat = pd.read_json('/content/drive/My Drive/Library_catalogue_preprocessed.jl', lines=True, orient='records').fillna('')
cat = cat[cat['interesting'] != '']
cat.replace({'interesting':'yes'}, 1, inplace=True)
cat.replace({'interesting':'no'}, 0, inplace=True)
cat.replace({'language':'pl'}, 1, inplace=True)
cat.replace({'language':'en'}, 0, inplace=True)

def join_tokens(row):
    return ' '.join(row['tokens'])

cat['tokens'] = cat.apply(join_tokens, axis=1)
cat.head()

,title,author,storage,WD_signature,publisher,year,pages,language,tokens,interesting
9,Mit sztuk zen w kształtowaniu się kultury arty...,"Sokołowski, Michał Andrzej.",magazyn,BF576 .B37165 2020,Kirin,2020,654,1,mita sztuka zen kształtowaniu kultura artystyc...,0
160,Magistrala CAN od A do Z : diagnostyka i progr...,"Kardaś, Paweł.",magazyn,TJ223.M53 K37 2020,Wydawnictwo Atnel,2020,204,1,magistrala can diagnostyk programowanie język c,0
221,Japoński socjolekt młodzieżowy jako manifestac...,"Duc-Harada, Patrycja.",,,Wydawnictwo Uniwersytetu Jagiellońskiego,2020,358,1,japoński socjolekt młodzieżowy manifestacja św...,0
279,Czysty kod w Pythonie,"Kapil, Sunil.",magazyn,QA76.73.P98 K365165 2020,Helion SA,2020,176,1,czysty koda pythonie,1
295,Żony nazistów : kobiety kochające zbrodniarzy,"Wyllie, James.",,DD245 .W85165 2020,Wydawnictwo Agora,2020,359,1,żona nazista kobieta kochać zbrodniarz,0


In [69]:
X = cat[['year', 'pages', 'language', 'tokens']]
y = cat['interesting']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scaler = MinMaxScaler().fit(X_train[['year', 'pages', 'language']])
X_train_scaled = scaler.transform(X_train[['year', 'pages', 'language']])
X_test_scaled = scaler.transform(X_test[['year', 'pages', 'language']])
cvect = CountVectorizer()
X_train_counts = cvect.fit_transform(X_train['tokens'])
X_test_counts = cvect.transform(X_test['tokens'])

tfidf = TfidfTransformer(use_idf=False)
X_train_transformed = tfidf.fit_transform(X_train_counts)
X_test_transformed = tfidf.transform(X_test_counts)

In [109]:
X_train_array = X_train_transformed.toarray()
X_test_array = X_test_transformed.toarray()

X_final = np.hstack((X_train_array, X_train_scaled))
X_final_test = np.hstack((X_test_array, X_test_scaled))
for iteration in range(50):
    X_final = np.hstack((X_final, X_train_scaled))
    X_final_test = np.hstack((X_final_test, X_test_scaled))

In [110]:
best_k = 0
best_acc = 0

for k in range(1,15):
    knn = KNeighborsClassifier(n_neighbors=k).fit(X_final, y_train)
    acc = accuracy_score(y_test, knn.predict(X_final_test))
    if acc > best_acc:
        best_acc = acc
        best_k = k

print(f'The highest accuracy is {best_acc}, for k == {k}')

The highest accuracy is 0.8004385964912281, for k == 14


In [111]:
logreg = LogisticRegression().fit(X_final, y_train)
accuracy_score(y_test, logreg.predict(X_final_test))

0.7719298245614035

In [112]:
svc = SVC(kernel='linear', C=1).fit(X_final, y_train)
accuracy_score(y_test, svc.predict(X_final_test))

0.8135964912280702

In [113]:
svc_rbf = SVC(kernel='rbf', gamma=1).fit(X_final, y_train)
accuracy_score(y_test, svc_rbf.predict(X_final_test))

0.7587719298245614

In [117]:
multi_nb = MultinomialNB(alpha=0.1).fit(X_train_transformed, y_train)
accuracy_score(y_test, multi_nb.predict(X_test_transformed))

0.8114035087719298

In [114]:
gauss_nb = GaussianNB().fit(X_final, y_train)
accuracy_score(y_test, gauss_nb.predict(X_final_test))

0.7412280701754386